# Chat with Audio Locally: A Guide to RAG with Whisper, Ollama, and Chromadb(can also use FAISS)
Features
1. Featured timestamp attached detection, for timestamp speech slice trace
2. manual cosine similarity search for audio
3. vector store similarity fetch docs for QA

Inspired by: 
* https://medium.com/@ingridwickstevens/chat-with-your-audio-locally-a-guide-to-rag-with-whisper-ollama-and-faiss-6656b0b40a68
* https://www.youtube.com/watch?v=TdMkKvzPe3E

### 1. Transcribe audio to text

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from progress_bar_decorator import progress_bar

In [2]:
has_mps = torch.backends.mps.is_available()
has_cuda = torch.cuda.is_available()
device = "mps" if has_mps else "cuda" if has_cuda else "cpu"
torch_dtype = torch.float16 if has_mps else torch.float32
device, torch_dtype

('mps', torch.float16)

In [3]:
model_id = "openai/whisper-large-v3"
# model_id = "openai/whisper-medium"

hf_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True, 
    use_safetensors=True,
    cache_dir='/Users/leon/Documents/03.LLM/whisper/models/'
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
pipe = pipeline(
    task="automatic-speech-recognition",
    model=hf_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,  # 128
    chunk_length_s=64,   # 30 
    batch_size=24,       # 16  
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    ignore_warning=True,
)

In [5]:
%%time
audio_file = './whisper/audio/Pinf_meeting.mp3'

@progress_bar(expected_time=180)
def transcribe():
    result = pipe(
        audio_file, 
        generate_kwargs={"language": "Mandarin",},
        return_timestamps=True,
    )
    return result

result = transcribe()

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [02:58<00:00,  1.79s/it]

CPU times: user 1min 5s, sys: 14.6 s, total: 1min 20s
Wall time: 2min 58s


In [53]:
# result['text']

In [7]:
import pandas as pd
df_transcribe = pd.DataFrame(result['chunks'])
df_transcribe

,timestamp,text
0,"(0.0, 1.84)",好的
1,"(1.84, 3.38)",看得到
2,"(3.38, 5.24)",OK 看得到
3,"(5.24, 6.58)",首先
4,"(6.58, 9.3)",欢迎大家
...,...,...
615,"(3689.05, 3723.06)",可以给到大家可能会有一个下一次我们再来review的一个action item然后那么咱们今...
616,"(3723.06, 3725.16)",然后谢谢大家的时间
617,"(3725.16, 3727.64)",好谢谢大家
618,"(3727.64, 3728.34)",谢谢


In [8]:
# parse timestamp function
def parse_audio_slice_timestamp(time_tuple):
    time_list = list(time_tuple)
    return time_list[0], time_list[1]

In [9]:
transcribe_filename = './whisper/transcribe/huggingface_Pinf_meeting.csv'

df_transcribe.loc[:, 'start'] = df_transcribe['timestamp'].apply(lambda x: list(x)[0])
df_transcribe.loc[:, 'end'] = df_transcribe['timestamp'].apply(lambda x: list(x)[1])
df_transcribe.to_csv(transcribe_filename, index=False)
df_transcribe.head()

,timestamp,text,start,end
0,"(0.0, 1.84)",好的,0.00,1.84
1,"(1.84, 3.38)",看得到,1.84,3.38
2,"(3.38, 5.24)",OK 看得到,3.38,5.24
3,"(5.24, 6.58)",首先,5.24,6.58
4,"(6.58, 9.3)",欢迎大家,6.58,9.30


In [10]:
transcribe_text_filename = './whisper/transcribe/Pinf_meeting.txt'

with open(transcribe_text_filename, 'w', encoding='utf-8') as f:
    f.write(result['text'])

In [12]:
from pydub import AudioSegment
from pydub.playback import play

sound = AudioSegment.from_file(audio_file)
print(f'Length of this audio file {round(len(sound)/1000/60, 2)} minutes')

row = df_transcribe.iloc[150, :]
print('Text:', row['text'])
print('Playing audio slice start from {}m to {}m'.format(row['start']/60, row['end']/60))

# audio timestamp in ms, hence times 1000
play(sound[row['start']*1000: row['end']*1000])

Length of this audio file 62.24 minutes
Text: 可能要更多的考虑到
Playing audio slice start from 14.535166666666667m to 14.565499999999998m


Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmpl14tettp.wav':
  Duration: 00:00:01.82, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


In [13]:
# play(sound[-1000:])

### 2. Tokenize and embed text

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser

#### 2.1 Direct embedding against audio

In [15]:
transcribe_filename = 'huggingface_Pinf_meeting.csv'

df_embed = pd.read_csv(transcribe_filename)
df_embed.head()

,timestamp,text,start,end
0,"(0.0, 1.84)",好的,0.00,1.84
1,"(1.84, 3.38)",看得到,1.84,3.38
2,"(3.38, 5.24)",OK 看得到,3.38,5.24
3,"(5.24, 6.58)",首先,5.24,6.58
4,"(6.58, 9.3)",欢迎大家,6.58,9.30


##### Choose embedding model

In [16]:
from langchain.embeddings import OllamaEmbeddings, SentenceTransformerEmbeddings
# embeddings = OllamaEmbeddings(model='llama2-chinese:latest')
# embeddings = OllamaEmbeddings(model='mxbai-embed-large:latest')
# embeddings = OllamaEmbeddings(model='nomic-embed-text:latest')

embeddings = SentenceTransformerEmbeddings(
    model_name='BAAI/bge-large-zh-v1.5', 
    cache_folder='/Users/leon/Documents/03.LLM/embedding_models'
)

In [17]:
# Lambda function to embed audio text
add_embed = lambda x: embeddings.embed_query(x['text'])

In [18]:
# similiarity search function
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [19]:
%%time
df_embed.loc[:, 'text_embed'] = df_embed.apply(add_embed, axis=1)
df_embed.head()

CPU times: user 13.5 s, sys: 1.64 s, total: 15.2 s
Wall time: 16.4 s


,timestamp,text,start,end,text_embed
0,"(0.0, 1.84)",好的,0.00,1.84,"[0.028062285855412483, 0.03217782825231552, -0..."
1,"(1.84, 3.38)",看得到,1.84,3.38,"[0.058578137308359146, -0.010690493509173393, ..."
2,"(3.38, 5.24)",OK 看得到,3.38,5.24,"[0.06891053169965744, -0.022434687241911888, 0..."
3,"(5.24, 6.58)",首先,5.24,6.58,"[-0.021473880857229233, 0.016622141003608704, ..."
4,"(6.58, 9.3)",欢迎大家,6.58,9.30,"[0.04682959243655205, -0.011324206367135048, -..."


In [20]:
# check embeded vector length
len(df_embed['text_embed'].iloc[0])

1024

In [21]:
# give your search query
search_term = '这个月的performance rating是什么'
search_term_embed = embeddings.embed_query(search_term)
len(search_term_embed)

1024

In [52]:
# conduct similiarity and sorting
df_embed.loc[:, 'cosine_similarity'] = df_embed['text_embed'].apply(lambda x: cosine_similarity(x, search_term_embed))
df_sorted = df_embed.sort_values(by='cosine_similarity', ascending=False)
df_sorted.head(3)

,timestamp,text,start,end,text_embed,cosine_similarity
15,"(53.47, 55.97)",然后我们对整个的这个performance rating,53.47,55.97,"[0.03247657045722008, 0.02801506407558918, -0....",0.628719
87,"(485.77, 488.01)",关于performance monitor的一项要求,485.77,488.01,"[0.022812873125076294, -0.010444370098412037, ...",0.579513
39,"(186.35, 189.15)",performance monitoring的维度上,186.35,189.15,"[0.023510172963142395, 0.008555696345865726, -...",0.577905


In [23]:
# playsound for top 5 ranking
for index, row in df_sorted.iloc[:5].iterrows():
    play(sound[row.start*1000: row.end*1000])

Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmp2q5657_a.wav':
  Duration: 00:00:02.50, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmpgvuahzt7.wav':
  Duration: 00:00:02.24, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmp5bqof1c6.wav':
  Duration: 00:00:02.80, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmp1u5hoylt.wav':
  Duration: 00:00:03.02, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


Input #0, wav, from '/var/folders/lv/4kql5s856s56ycnzm1ly8y0m0000gn/T/tmppwfo4p7t.wav':
  Duration: 00:00:33.36, bitrate: 768 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s16, 768 kb/s


#### 2.2 Embedding for LLM-based RAG

In [24]:
# define text to split
with open(transcribe_text_filename, 'r') as f:
    transcribe_text = f.read()

# split the text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = splitter.split_text(transcribe_text)

In [25]:
len(texts), texts[-2]

(28,
 'Launch是在二三年的十二月份它主要的一个开发功能是包括Pipeline管理工作管理和客户管理等等它风险是目前进入上是没有已经实施已经暂停那现在业务正在探索其他的解决方案来降低成本那一旦就是确定了解决方案IT工作就会恢复那后期如果有最新的进展下次会议会同步给大家就是最新的Money Poly的一个最新状况最后一个是关于审测审测就是说它的SDK定到iHub的数据传输的实施也是正在进行中以上是关于聘服的项目进展情况谢谢大家谢谢上门包括Leo还有Jene如果比如说你们对哪个项目比较关注或者说你觉得这个项目可能比较重要的话在后面比如说以后汇报或者说更近的时候就会更多的去帮大家去这个project list对就是可能先先听一下大家的反馈吧这对这个项目当然我们之前提到的就是把这些项目我们开成两页包括这个信息的规范我们这个下次一定会改进好的好')

In [26]:
################################
# speech_vector.delete_collection()

In [29]:
# create vector store using Chroma
speech_vector = Chroma.from_texts(
    texts, 
    embedding=embeddings, 
    metadatas=[{'source': str(i)} for i in range(len(texts))],
    collection_name='speech-rag',
)

### 3.Setup LLM and Prompt

In [30]:
!ollama list

NAME                         	ID          	SIZE  	MODIFIED     
command-r:35b-v0.1-q6_K      	c46e949ec735	28 GB 	3 days ago  	
llama2:13b-f16               	18051f2e82e3	26 GB 	2 weeks ago 	
llama2:7b-f32                	4901050728fc	26 GB 	2 weeks ago 	
llama2-chinese:13b-chat-fp16 	3d4c5a00962c	26 GB 	2 weeks ago 	
llama2-chinese:7b-chat-fp16  	b73150f2949c	13 GB 	2 weeks ago 	
llama3:70b-instruct-q4_0     	bcfb190ca3a7	39 GB 	38 hours ago	
llama3:8b-instruct-fp16      	c1d0ea97005c	16 GB 	39 hours ago	
llama3:8b-text-fp16          	fc1ae0909d51	16 GB 	39 hours ago	
llava:34b-v1.6-q6_K          	8f572ea02185	28 GB 	3 days ago  	
mistral:7b-instruct-v0.2-fp16	094d67ff087c	14 GB 	3 days ago  	
mixtral:latest               	7708c059a8bb	26 GB 	2 weeks ago 	
mxbai-embed-large:latest     	468836162de7	669 MB	11 days ago 	
nomic-embed-text:latest      	0a109f422b47	274 MB	2 weeks ago 	
wizardlm2:7b-fp16            	a34a3bbd552b	14 GB 	4 days ago  	


In [31]:
from langchain.llms import Ollama

# setup llm
# local_llm = 'llama3:8b-instruct-fp16'
# local_llm = 'command-r:35b-v0.1-q6_K'
# local_llm = 'wizardlm2:7b-fp16'
# local_llm = 'mistral:7b-instruct-v0.2-fp16'
local_llm = 'mixtral:latest'

llm = Ollama(model=local_llm)

In [32]:
# from langchain_community.llms.chatglm3 import ChatGLM3

# llm = ChatGLM3(
#     model='chatglm3-6b',
#     endpoint_url='http://127.0.0.1:8000/v1/chat/completions',
#     verbose=True
# )
# llm.invoke('你好')

In [33]:
# setup prompt
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [34]:
# create RAG prompt
rag_prompt = ChatPromptTemplate(
    input_variables=['context', 'question'],
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                # template="""You answer questions about the contents of a transcribed audio file.
                # Use only the provided audio file transcription as context to answer the question. 
                # Do not use any additional information.
                # If you don't know the answer, just say that you don't know. Do not use external knowledge. 
                # Use three sentences maximum and keep the answer concise. 
                # Make sure to reference your sources with quotes of the provided context as citations.
                # \nQuestion: {question} \nContext: {context} \nAnswer:
                # """,
                template="""你针对会议录音转的文字内容回答问题。
                只利用录音转的文字内容作为上下文来回答问题。
                不要使用任何其它额外信息。
                如果你不知道答案，就回答不知道，不要使用外部知识。
                用最多五句话来回答，并确保答案准确。
                确保在答案中对上下文的源信息进行引用。
                \nQuestion: {question} \nContext: {context} \nAnswer:
                """
            )
        )
    ]
)

In [35]:
# load qa chain
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm=llm, chain_type='stuff', prompt=rag_prompt, verbose=False)

### Query and Answering

In [36]:
# setup a query
query = '这次Performance review的rating是什么？'
# query = '监管政策的解读'

In [37]:
# similarity search
# docs = speech_vector.max_marginal_relevance_search(query, k=5, fetch_k=28, lambda_mult=0.5)
docs = speech_vector.similarity_search(query, )
# docs

In [38]:
# using chain for the query
response = chain.invoke(
    input={'input_documents': docs, 'question': query}, 
    # return_only_outputs=True,
)

# Display the response
print("Based on the provided context, the self-evident propositions in the speech are:")
# print("\n".join(response["output_text"]))
print(response["output_text"])

Based on the provided context, the self-evident propositions in the speech are:
 Based on the meeting recording transcript, the exact rating from the performance review was not explicitly stated. However, it was mentioned that there were improvements from December to January, with December having an amber rating in the performance monitoring dimension and January showing all green ratings, except for a failed PLA agreed service. Without specific reference to the rating, it can be inferred that the review result improved from December to January.


In [39]:
print(chain.invoke({'input_documents': docs, 'question': query},)['output_text'])

 Based on the context provided, the exact performance review rating is not explicitly mentioned. However, it's stated that there was an 'amber' rating in December, which was categorized under the 'performance monitoring' dimension. In January, the situation improved as everything was 'green', except for the PLA service that failed. So, without a clear rating for the performance review, I can only report the status of the 'performance monitoring' metric, which was 'amber' in December and not explicitly mentioned for January.


In [44]:
from langchain_community.chat_models import ChatOllama

# setup llm
local_llm = 'wizardlm2:7b-fp16'
# local_llm = 'llama3:8b-instruct-fp16'
llm = ChatOllama(model=local_llm)

In [45]:
# get retriever --> equvalent to vector search
retriever = speech_vector.as_retriever(
    search_type='similarity',  # similarity, mmr, similarity_score_threshold
    search_kwargs={'k':5, },  # k, score_threshold
)

# check retriever
docs = retriever.get_relevant_documents(query)
# docs

In [46]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [47]:
print(chain.invoke(query))

根据会议录音转换的文字内容，Performance review的rating在1月份的评估中表现为全部green，即良好状态。在12月份的评估中，amber rating（黄色状态）被放置在performance monitoring（性能监控）的维度上。这说明在性能监控方面可能存在需要改进的地方。总的来说，这次的Performance review是为了评估外包服务的表现，并且提供了一个lesson learned，即在进行系统变更时需要更多地考虑到对下游周边系统的影响以及沟通。具体的rating metrics可以在相关文档中查看，其中包括多个维度的综合评估。在1月份的最新deck中，也有记录了这些问题作为problems需要解决。

引用来源：提供的文档内容（页面内容1, 6, 1）。


#### 2.4 Meeting Minutes Summary

In [48]:
type(transcribe_text)

str

In [49]:
# create summary prompt
summary_prompt_template = """Your goal is to summarize the meeting transcription that is given to you as the following:
                "{text}"
                The summarization of the meeting minutes shall limit to 2500 words.
                Only output the summary without any additional text.
                Focus on providing a summary in a structured format text of what subject reviewed and the action items out of it.
                """
summary_prompt = PromptTemplate.from_template(summary_prompt_template)


In [51]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

docs = [Document(page_content=transcribe_text, metadata={"source": "local"})]

llm_chain = LLMChain(llm=llm, prompt=summary_prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

# print(stuff_chain.invoke(docs)['output_text'])
